# 30dim Branin Benchmark with SAASBO
This is a port from https://github.com/pytorch/botorch/blob/main/tutorials/saasbo.ipynb
## Imports

In [1]:
from bofire.benchmarks.single import Branin30
from bofire.benchmarks.benchmark import run
from bofire.data_models.strategies.api import SoboStrategy, RandomStrategy, PolytopeSampler
from bofire.data_models.surrogates.api import SaasSingleTaskGPSurrogate, BotorchSurrogates
from bofire.data_models.acquisition_functions.api import qEI
import bofire.strategies.api as strategies
from bofire.data_models.api import Domain
from functools import partial
import pandas as pd

/opt/homebrew/Caskroom/miniforge/base/envs/bofire/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Random Optimization

In [9]:
def sample(domain):
    datamodel = PolytopeSampler(domain=domain)
    sampler = strategies.map(data_model=datamodel)
    sampled = sampler.ask(10)
    return sampled

def best(domain: Domain, experiments: pd.DataFrame) -> float:
    return experiments.y.min()

random_results = run(
    Branin30(),
    strategy_factory=RandomStrategy,
    n_iterations=10,
    metric=best,
    initial_sampler=sample,
    n_candidates_per_proposal=5,
    n_runs=1,
    n_procs=1,
)

run 00 with current best 2.097: 100%|██████████| 10/10 [00:00<00:00, 39.90it/s]


## SAASBO Optimization

In [10]:
benchmark = Branin30()

random_results = run(
    Branin30(),
    strategy_factory=partial(
        SoboStrategy, 
        acquisition_function=qEI(),
        surrogate_specs=BotorchSurrogates(
            surrogates=[
                SaasSingleTaskGPSurrogate(input_features=benchmark.domain.inputs, output_features=benchmark.domain.outputs)])
    ),
    n_iterations=10,
    metric=best,
    initial_sampler=sample,
    n_candidates_per_proposal=5,
    n_runs=1,
    n_procs=1,
)

run 00 with current best 0.399: 100%|██████████| 10/10 [10:10<00:00, 61.05s/it]
